In [1]:
from __future__ import print_function, division
import numpy as np
import pprint
import os 
import fnmatch
import os.path as osp
import json
import glob as gb
from six import string_types
from nipype.interfaces.base import Bunch

import utils
reload(utils)

<module 'utils' from 'utils.pyc'>

In [2]:
pp = pprint.PrettyPrinter(indent=2)

In [3]:
#- testing

test_case_105a = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_level-run_sub*_model.json"
}
test_case_105b = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_level-all_model.json"
}
test_case_005 = {
    'base_dir': '/home/jb/data/bids/ds005',
    'model_pattern': "*_model.json"
}
test_case_005r = {
    'base_dir': '/home/jb/data/bids/ds005',
    'model_pattern': "ds-005_type-russ*_model.json"
}

test_case_005t = {
    'base_dir': '/home/jb/code/bids2pype/bids2pype/tests/data/ds005',
    'model_pattern': "ds-005_type-russ*_model.json"
}


#test_case = test_case_005
#test_case = test_case_005r
test_case = test_case_005t

#test_case = test_case_105a
#test_case = test_case_105b

base_dir, model_pattern = test_case['base_dir'], test_case['model_pattern'] 
print(test_case) 

{'model_pattern': 'ds-005_type-russ*_model.json', 'base_dir': '/home/jb/code/bids2pype/bids2pype/tests/data/ds005'}


## Association between data and model 

In [4]:
assos_model_data = utils.associate_model_data(base_dir, model_pattern, 
                                                  verbose=utils.VERB['none'])
data_dict = assos_model_data['data_dict']

#print(assos_model_data)
datafile = sorted(data_dict.keys())[0]
print(datafile)
pp.pprint(data_dict[datafile]['Contrasts'].keys())

/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz
[u'param-loss-neg', u'param-gain', u'task>Baseline']


## Read data and export as nipype data structures

In [5]:
model_dict = data_dict[datafile]
utils._get_nipype_contrasts(model_dict)

[(u'param-loss-neg', u'T', [u'param-loss'], [-1]),
 (u'task>Baseline', u'T', [u'task'], [1]),
 (u'param-gain', u'T', [u'param-gain'], [1])]

In [6]:
mvt = np.loadtxt(base_dir + '/derivatives/mcflirt/par/_runcode_1/_subject_id_sub-01/' + \
                                    'sub-01_task-mixedgamblestask_run-01_bold_mcf.nii.gz.par')

In [7]:
mvt[:10,0]


array([ 0.00147019,  0.00082571,  0.00144829,  0.00105914,  0.00170214,
        0.00131321,  0.00124525,  0.00143436,  0.00138788,  0.00097287])

In [8]:
specifymodel_inputs, bunches, data = \
    utils._get_nipype_specify_model_inputs(base_dir, model_pattern, bunch_type='fsl', 
                                                                 verbose=utils.VERB['none'])

    
sorted_bunch = [b for (d,b) in sorted(zip(data, bunches))]
sorted_data = sorted(data)
        
#print(specifymodel_inputs)
#print('\n')
#print(data[0])
print('\n')
print(sorted_data[0]) 
print(sorted_data[1]) 
print(sorted_data[-1]) 

print('conditions:', sorted_bunch[0].conditions)
#print(sorted_bunch[0].conditions[3])
#print(sorted_bunch[0].amplitudes[3])
#print(index_gain)
print((sorted_bunch[0].regressor_names))
print('Regressors shape:', sorted_bunch[0].regressors[0].shape)
print((sorted_bunch[0].regressors[0][:33]))
print((sorted_bunch[0].regressors[1][:33]))




/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz
/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-02_bold.nii.gz
/home/jb/code/bids2pype/bids2pype/tests/data/ds005/sub-16/func/sub-16_task-mixedgamblestask_run-03_bold.nii.gz
conditions: [u'task', u'param-loss', u'param-rt', u'param-gain']
[u'motion-param_01', u'motion-param_02', u'motion-param_03', u'motion-param_04', u'motion-param_05', u'motion-param_06', u'motion-param_01_deriv', u'motion-param_02_deriv', u'motion-param_03_deriv', u'motion-param_04_deriv', u'motion-param_05_deriv', u'motion-param_06_deriv']
Regressors shape: (240,)
[ 0.00147019  0.00082571  0.00144829  0.00105914  0.00170214  0.00131321
  0.00124525  0.00143436  0.00138788  0.00097287  0.00119608  0.00086367
  0.00112919  0.00086367  0.00124277  0.00086367  0.00147362  0.00093246
  0.0011265   0.00094951  0.00114544  0.00061997  0.00053421  0.00073343
  0.00

### copy a ds tree with only the json and tsv files

In [9]:
source_dir = '/home/jb/data/bids/ds005'
dest_dir = '/home/jb/code/bids2pype/bids2pype/tests/data/ds005'
#utils.create_empty_bids(source_dir, dest_dir, ['*.json','*.tsv', '*.par'])